In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import matplotlib.pylab as plt

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(1)
if device == 'cuda':
    torch.cuda.manual_seed_all(1)

In [3]:
learning_rate = 0.01
training_epochs = 10
batch_size = 32

In [4]:
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)
mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

SyntaxError: invalid syntax (<ipython-input-4-76985d36ede1>, line 4)

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                           batch_size=batch_size,
                                           shuffle=True,
                                           drop_last=True)
test_loader = torch.utils.data.DataLoader(dataset=mnist_test,
                                          batch_size=batch_size,
                                          shuffle=False,
                                          drop_last=True)

## Batch Normalization
<br></br>
신경망의 각 layer에서 데이터(배치)의 분포를 정규화하는 작업

데이터가 레이어를 거치는 과정에서 분포가 처음에 비해 왜곡되는 문제를 해결할 수 있다

In [ ]:
linear1 = torch.nn.Linear(784, 32, bias=True)
linear2 = torch.nn.Linear(32, 32, bias=True)
linear3 = torch.nn.Linear(32, 10, bias=True)
relu = torch.nn.ReLU()
bn1 = torch.nn.BatchNorm1d(32)
bn2 = torch.nn.BatchNorm1d(32)

In [ ]:
# batch norm model
bn_model = torch.nn.Sequential(linear1, bn1, relu,
                               linear2, bn2, relu,
                               linear3).to(device)
# no batch norm model
nn_model = torch.nn.Sequential(linear1, relu,
                               linear2, relu,
                               linear3).to(device)

In [ ]:
# loss / batch norm optimizer / no batch norm optimizer
criterion = torch.nn.CrossEntropyLoss().to(device)
bn_optimizer = torch.optim.Adam(bn_model.parameters(), lr=learning_rate)
nn_optimizer = torch.optim.Adam(bn_model.parameters(), lr=learning_rate)

In [ ]:
# back propagation
def backprop(X, Y, model, optimizer) -> None:
    hypothesis = model(X)
    cost = criterion(hypothesis, Y)
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

In [ ]:
# test model and return loss, accuracy
def test_model(model, loader, total_batch) -> float:
    loss, acc = 0, 0
    
    with torch.no_grad():
        for i, (X, Y) in enumerate(loader):
            X = X.view(-1, 28 * 28).float().to(device)
            Y = Y.to(device)
            
            prediction = model(X)
            correct_prediction = torch.argmax(prediction, 1) == Y
            loss += criterion(prediction, Y)
            acc += correct_prediction.float().mean()
        
        loss, acc = loss / total_batch, acc / total_batch
        return loss, acc

In [ ]:
# print and save Trainset loss/acc vs Test(Valid)set loss/acc
def print_save_result(epoch, loader, total_batch, losses, accs, t_or_v) -> None:
    bn_loss, bn_acc = test_model(bn_model, loader, total_batch)
    nn_loss, nn_acc = test_model(nn_model, loader, total_batch)
    
    losses.append([bn_loss, nn_loss])
    accs.append([bn_acc, nn_acc])
    print('[Epoch %d-' % (epoch + 1) + t_or_v\
          + '] Batchnorm Loss(Acc): bn_loss:%.5f(bn_acc:%.2f)\n\
          vs No Batchnorm Loss(Acc): nn_loss:%.5f(nn_acc:%.2f)'\
          % (bn_loss.item(), bn_acc.item(), nn_loss.item(), nn_acc.item()))

In [ ]:
# save all train loss data and accuracy data
train_losses = []
train_accs = []

valid_losses = []
valid_accs = []

train_total_batch = len(train_loader)
test_total_batch = len(test_loader)
for epoch in range(training_epochs):
    bn_model.train()  # set batch norm model to train mode
    
    for X, Y in train_loader:
        X = X.view(-1, 28 * 28).float().to(device)
        Y = Y.to(device)
        
        # back propagation
        backprop(X, Y, bn_model, bn_optimizer)
        backprop(X, Y, nn_model, nn_optimizer)
    
    bn_model.eval()  # set batch norm model to evaluation mode
    
    # print and save Trainset loss/acc vs Test(Valid)set loss/acc
    print_save_result(epoch, train_loader, train_total_batch,
                      train_losses, train_accs, 'TRAIN')
    print_save_result(epoch, test_loader, test_total_batch,
                      valid_losses, valid_accs, 'VALID')
    
print('\nLearning finished')

In [ ]:
def plot_compare(loss_list: list, ylim=None, title=None) -> None:
    bn = [i[0] for i in loss_list]
    nn = [i[1] for i in loss_list]
    
    plt.figure(figsize=(15, 10))
    plt.plot(bn, label='With BN')
    plt.plot(nn, label='Without BN')
    
    if ylim:
        plt.ylim(ylim)
        
    if title:
        plt.title(title)
    
    plt.legend()    # 범례 추가
    plt.grid('on')  # 격자선 추가
    plt.show()

In [ ]:
plot_compare(train_losses, title='Training Loss at Epoch')
plot_compare(train_accs, [0, 1.0], title='Training Acc at Epoch')
plot_compare(valid_losses, title='Validation Loss at Epoch')
plot_compare(valid_accs, [0, 1.0], title='Validation Acc at Epoch')